In [1]:
!pip install transformers datasets accelerate torch
!pip install arabert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import necessary libraries
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [3]:
dataset=pd.read_csv('all_data_meta.csv',sep=None, engine='python', on_bad_lines='skip')

In [4]:
dataset

,answer_id,question_id,text,answer_start,answer_end,answer_category,question,file_name,open-domain,long,Unnamed: 10,Unnamed: 11
0,649960,649960,المعلومات المخزنة أو وسيلة التخزين.\nأي أن الب...,251,418,NaN,اذكر وسيله التخزين ؟,1338943.0,1,1.0,1 long 0 short,3167
1,649961,649961,يتكون البايت عادة من 8 بت، ولذلك فأن البايت ي...,417,536,NaN,مما يتكون البايت ؟,1338943.0,1,1.0,1 open 0 no internet,202
2,649962,649962,لاحظ أن الأسماء كيلوبايت وميجابايت ...الخ، يمك...,748,874,NaN,ما هي مضاعفات البايت ؟,1338943.0,1,1.0,NaN,NaN
3,649963,649963,وبتمديد النمط، نستطيع الحصول على وحدتين إضافيت...,882,1015,NaN,ما هو الاستخدام البايت ؟,1338943.0,1,1.0,NaN,NaN
4,649959,649959,البايت أو الثُّمَانِيَّة هي وحدة معلومات رقمية...,0,233,NaN,ما هو البايت ف الحاسوب ؟,1338943.0,1,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
60646,1191158,1191158,ريو دي جانيرو في البرازيل,70,95,NaN,أين ولد واشنطن ؟,1717957.0,واشنطن هو لاعب كرة قدم برازيلي في مركز الهجوم،...,1.0,0,None
60647,1191159,1191159,بوتافوغو ريغاتاس.,104,121,NaN,ما هو النادي الذي لعب معه واشنطن ؟,1717957.0,واشنطن هو لاعب كرة قدم برازيلي في مركز الهجوم،...,1.0,0,None
60648,1196592,1196592,تعود إلى عصور ما قبل التاريخ ونقوش صخرية تعود...,111,216,NaN,متى تعود النقوش الصخرية في منطقة الجلفة إلى؟\n,1717629.0,الفن الصخري بمنطقة الجلفة تتكون النقوش الصخرية...,1.0,1,None
60649,1196593,1196593,في الشرق في منطقة قسنطينة (الجزائر),350,386,NaN,ما هي المناطق الأخرى في الجزائر التي وُجِدت في...,1717629.0,الفن الصخري بمنطقة الجلفة تتكون النقوش الصخرية...,0.0,0,None


In [5]:
pip install torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 14.0 MB/s eta 0:00:00


In [6]:
pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


# 1. Fine-tune the target model (see table below) on your training dataset and assess the performance on the test dataset

In [ ]:
import os
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset

# Disable WandB (optional)
os.environ["WANDB_DISABLED"] = "true"

# Load metadata
metadata = pd.read_csv("/content/all_data_meta.csv", on_bad_lines='skip')

# Combine question and text columns
metadata['text'] = metadata['question'].fillna('') + " " + metadata['text'].fillna('')

# Save prepared text data
text_file_path = "finetune_data.txt"
with open(text_file_path, "w", encoding="utf-8") as f:
    for line in metadata['text']:
        f.write(line.strip() + "\n")

# Load AraGPT2 model and tokenizer
model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add special tokens (if necessary)
tokenizer.pad_token = tokenizer.eos_token

# Load the dataset
dataset = load_dataset('text', data_files={'train': text_file_path})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Split the dataset into training and evaluation
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Define a data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="aragpt2-finetuning",  # Custom run name
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="epoch",
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,  # Use data collator
)

# Train the model
trainer.train()

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/108094 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss


tester avec F1 Score and EM

In [ ]:
from datasets import load_metric
metric = load_metric("squad")

# Compute EM and F1
def compute_metrics(p):
    predictions, references = p
    results = metric.compute(predictions=predictions, references=references)
    return results

# Example for evaluating on a dataset
results = compute_metrics((generated_answers, reference_answers))
print(f"EM: {results['exact_match']}")
print(f"F1 Score: {results['f1']}")


load model

In [ ]:
# Define the path where you want to save the model and tokenizer
save_path = "./fine_tuned_aragpt2"

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_aragpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_aragpt2")


# 2. Implement a multi-head attention class based on differential attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DifferentialAttention(nn.Module):
    def __init__(self, d_model, num_heads, d_head, lambda_init=0.8):
        super(DifferentialAttention, self).__init__()
        assert d_model % (2 * num_heads) == 0, "d_model must be divisible by 2 * num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_head = d_head
        self.lambda_init = lambda_init

        # Learnable weights for Q, K, V
        self.WQ = nn.Parameter(torch.randn(d_model, 2 * d_head))
        self.WK = nn.Parameter(torch.randn(d_model, 2 * d_head))
        self.WV = nn.Parameter(torch.randn(d_model, 2 * d_head))

        # Learnable scalars for λ
        self.lambda_q1 = nn.Parameter(torch.randn(d_head))
        self.lambda_k1 = nn.Parameter(torch.randn(d_head))
        self.lambda_q2 = nn.Parameter(torch.randn(d_head))
        self.lambda_k2 = nn.Parameter(torch.randn(d_head))

    def forward(self, X):
        N, d_model = X.shape

        # Compute Q1, Q2, K1, K2, V
        Q = torch.matmul(X, self.WQ)
        K = torch.matmul(X, self.WK)
        V = torch.matmul(X, self.WV)
        Q1, Q2 = Q[:, :self.d_head], Q[:, self.d_head:]
        K1, K2 = K[:, :self.d_head], K[:, self.d_head:]

        # Compute λ
        lambda_val = (
            torch.exp(self.lambda_q1 * self.lambda_k1)
            - torch.exp(self.lambda_q2 * self.lambda_k2)
            + self.lambda_init
        )

        # Differential Attention
        attn1 = F.softmax(torch.matmul(Q1, K1.T) / (self.d_head ** 0.5), dim=-1)
        attn2 = F.softmax(torch.matmul(Q2, K2.T) / (self.d_head ** 0.5), dim=-1)
        diff_attn = attn1 - lambda_val * attn2

        # Weighted sum over values
        output = torch.matmul(diff_attn, V)
        return output


In [ ]:
class MultiHeadDifferentialAttention(nn.Module):
    def __init__(self, d_model, num_heads, d_head, lambda_init=0.8):
        super(MultiHeadDifferentialAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_head = d_head

        # Create a DifferentialAttention module for each head
        self.heads = nn.ModuleList([
            DifferentialAttention(d_model, 1, d_head, lambda_init) for _ in range(num_heads)
        ])

        # Final linear layer to combine heads
        self.out_proj = nn.Linear(num_heads * d_head, d_model)

    def forward(self, X):
        head_outputs = [head(X) for head in self.heads]
        concatenated = torch.cat(head_outputs, dim=-1)  # Concatenate along feature dimension
        return self.out_proj(concatenated)


In [ ]:
# Replace in a transformer block
class CustomTransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_head, lambda_init=0.8):
        super(CustomTransformerBlock, self).__init__()
        self.diff_attention = MultiHeadDifferentialAttention(d_model, num_heads, d_head, lambda_init)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model),
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, X):
        attn_output = self.diff_attention(X)
        X = self.norm1(X + attn_output)
        ff_output = self.feed_forward(X)
        return self.norm2(X + ff_output)


In [ ]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model, GPT2Block
from torch.nn import Module

class CustomGPT2Block(GPT2Block):
    def __init__(self, config, use_diff_attention=False, diff_attn_params=None):
        super().__init__(config)
        self.use_diff_attention = use_diff_attention

        if use_diff_attention:
            d_model = config.hidden_size
            num_heads = config.num_attention_heads
            d_head = d_model // num_heads

            # Initialize the differential attention
            self.attn = MultiHeadDifferentialAttention(
                d_model=d_model,
                num_heads=num_heads,
                d_head=d_head,
                lambda_init=diff_attn_params.get("lambda_init", 0.8),
            )

    def forward(self, hidden_states, layer_past=None, attention_mask=None, use_cache=False):
        if self.use_diff_attention:
            attn_output = self.attn(hidden_states)
        else:
            attn_output = super().attn(hidden_states, layer_past, attention_mask, use_cache)
        return super().mlp(attn_output)


# 3. Using your fine-tuned model, replace between 25% to 50% of the encoder and/or decoder layers’ multihead attention with multi-head differential attention. For each configuration you propose, train the new model, and assess the performance

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel

def modify_gpt2_model(model, replace_fraction=0.25, lambda_init=0.8):
    """
    Modify the GPT-2 model by replacing a fraction of its attention layers with differential attention.
    """
    total_layers = len(model.transformer.h)
    num_layers_to_replace = int(total_layers * replace_fraction)
    layers_to_replace = sorted(range(total_layers))[-num_layers_to_replace:]  # Replace last layers

    for i in layers_to_replace:
        model.transformer.h[i] = CustomGPT2Block(
            config=model.config,
            use_diff_attention=True,
            diff_attn_params={"lambda_init": lambda_init},
        )
    return model

# Load the fine-tuned model and apply modifications
model_name = "./results"  # Path to fine-tuned model
model = GPT2LMHeadModel.from_pretrained(model_name)
modified_model = modify_gpt2_model(model, replace_fraction=0.25)


In [ ]:
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


# Fine-tune the modified model
trainer.train()

# Evaluate performance
results = trainer.evaluate()
print("Evaluation Results for Modified Model:", results)